In [54]:
import os 
import json
import logging
import requests
import pandas as pd
from time import sleep
from random import randint
from datetime import datetime, timedelta

logging.getLogger().setLevel(logging.INFO)


def strToFloat(pdf, num_cols):
    for col in num_cols:
        pdf[col] = pdf[col].astype(str).str.replace(',', '').astype(float)
    return pdf

def toRocYear(date_str, sep='/'):
    YEAR_OFFSET = 1911
    split = date_str.split(sep)
    ori_year = split[0]
    new_year = str(int(ori_year) - YEAR_OFFSET)
    new_date_str = date_str.replace(ori_year, new_year)
    return new_date_str


logging.info("checking output path")
output_path = "../data/institutional_investor"
if not os.path.exists(output_path):
    logging.info("creating output path")
    os.makedirs(output_path, exist_ok=True)

since_days_before = 30
last_update_path = output_path + os.sep + "lastUpated"
last_update = None
if not os.path.exists(last_update_path):
    logging.info("lastUpated is not existed")
    last_update_file = open(last_update_path, "w+")
    now = datetime.now()
    last_update_dt = (datetime.now() - timedelta(days=since_days_before)).strftime('%Y/%m/%d')
    last_update_file.write(last_update_dt)
    last_update_file.close()
else:
    last_update_file = open(last_update_path)
    last_update_dt = last_update_file.read()
    last_update_file.close()
    
start_dt = datetime.strptime(last_update_dt, '%Y/%m/%d') + timedelta(days=1)
end_dt = datetime.now().strftime('%Y/%m/%d')
dt = start_dt
end_dt = datetime.strptime(end_dt, '%Y/%m/%d')
logging.info(f"crawling {start_dt} ~ {end_dt}")

tpex_url = 'https://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_result.php?l=zh-tw&se=EW&t=D&d=%s'
twse_url = 'https://www.twse.com.tw/fund/T86?response=json&date=%s&selectType=ALLBUT0999'
tpex_header_old = ["股票代號","名稱","外陸資買進股數(不含外資自營商)","外陸資賣出股數(不含外資自營商)","外陸資買賣超股數(不含外資自營商)",
                   "投信買進股數","投信賣出股數","投信買賣超股數","自營商買賣超股數","自營商買進股數(自行買賣)","自營商賣出股數(自行買賣)",
                   "自營商買賣超股數(自行買賣)","自營商買進股數(避險)","自營商賣出股數(避險)","自營商買賣超股數(避險)",
                   "三大法人買賣超股數", "其他"]
tpex_header_new = ["股票代號","名稱","外陸資買進股數(不含外資自營商)","外陸資賣出股數(不含外資自營商)","外陸資買賣超股數(不含外資自營商)",
                   "外資自營商買進股數","外資自營商賣出股數","外資自營商買賣超股數","外資及陸資-買進股數","外資及陸資-賣出股數",
                   "外資及陸資-買賣超股數","投信買進股數","投信賣出股數","投信買賣超股數","自營商買進股數(自行買賣)",
                   "自營商賣出股數(自行買賣)","自營商買賣超股數(自行買賣)","自營商買進股數(避險)","自營商賣出股數(避險)",
                   "自營商買賣超股數(避險)","自營商買進股數","自營商賣出股數","自營商買賣超股數","三大法人買賣超股數", "其他"]
header_list = ['股票代號', '名稱','外資買賣超股數', '投信買賣超股數', '自營商買賣超股數', '自營商買賣超股數(避險)', '三大法人買賣超股數']
header_str = '股票代號,日期,外資買賣超股數,投信買賣超股數,自營商買賣超股數,自營商買賣超股數(避險),三大法人買賣超股數\n'
while dt <= end_dt:
    if dt.weekday() in range(0, 5):
        roc_dt = toRocYear(dt.strftime('%Y/%m/%d'))
        tpex_resp = requests.get(tpex_url % roc_dt)
        json_obj = json.loads(tpex_resp.text)
        tpex_data = json_obj['aaData']
        tpex_pdf = None
        try:
            tpex_pdf = pd.DataFrame(tpex_data, columns=tpex_header_new)
        except:
            tpex_pdf = pd.DataFrame(tpex_data, columns=tpex_header_old)
        tpex_pdf = tpex_pdf[tpex_pdf['股票代號'].str.len() == 4]
        tpex_pdf = tpex_pdf.rename(columns={'外陸資買賣超股數(不含外資自營商)': '外資買賣超股數'})
        tpex_pdf = tpex_pdf[header_list]
        
        twse_resp = requests.get(twse_url % dt.strftime('%Y%m%d'))
        if 'data' in twse_resp.text:
            json_obj = json.loads(twse_resp.text)
            twse_data = json_obj['data']
            twse_header = json_obj['fields']
            twse_pdf = pd.DataFrame(twse_data, columns=twse_header)
            twse_pdf = twse_pdf[twse_pdf['證券代號'].str.len() == 4]
            twse_pdf = twse_pdf.rename(columns={'證券代號':'股票代號', '證券名稱':'名稱', '外陸資買賣超股數(不含外資自營商)': '外資買賣超股數'})
            twse_pdf = twse_pdf[header_list]
        else:
            dt = dt + timedelta(days=1)
            sleep(randint(3, 5))
            continue
        
        pdf = tpex_pdf.append(twse_pdf)
        tmp_pdf = pdf[pdf['三大法人買賣超股數'].isna()].copy()
        pdf.loc[pdf['三大法人買賣超股數'].isna(), '三大法人買賣超股數'] = tmp_pdf['外資買賣超股數'] + tmp_pdf['投信買賣超股數'] + tmp_pdf['自營商買賣超股數']
        pdf.loc[(pdf['自營商買賣超股數(避險)'].isna()) & (pdf['自營商買賣超股數']==0), '自營商買賣超股數(避險)'] = 0
        pdf = strToFloat(pdf, ['外資買賣超股數', '投信買賣超股數', '自營商買賣超股數', '自營商買賣超股數(避險)', '三大法人買賣超股數'])

        for row in pdf.iterrows():
            data = row[1]
            stock_no = data['股票代號']
            foreign_institution = data['外資買賣超股數']
            domestic_institution = data['投信買賣超股數']
            dealer_institution = data['自營商買賣超股數']
            dealer_hedging = data['自營商買賣超股數(避險)']
            institutional_investors = data['三大法人買賣超股數']
            row = "{},{},{},{},{},{},{}\n".format(stock_no, dt.strftime('%Y-%m-%d'), foreign_institution, 
                                        domestic_institution, dealer_institution, dealer_hedging, institutional_investors)
            file_path = output_path + os.sep + stock_no + '.csv'
            
            if os.path.exists(file_path):
                file = open(file_path, 'a')
                file.write(row)
            else:
                file = open(file_path, 'w')
                file.write(header_str)
                file.write(row)
            file.close()
        logging.info("{} is done".format(dt.strftime('%Y/%m/%d')))
        last_update_file = open(last_update_path, "w+")
        last_update_file.write(dt.strftime('%Y/%m/%d'))
        last_update_file.close()
    dt = dt + timedelta(days=1)
    sleep(randint(2, 4))

INFO:root:checking output path
INFO:root:crawling 2021-06-24 00:00:00 ~ 2021-07-18 00:00:00
INFO:root:2021/06/24 is done
INFO:root:2021/06/25 is done
INFO:root:2021/06/28 is done
INFO:root:2021/06/29 is done
INFO:root:2021/06/30 is done
INFO:root:2021/07/01 is done
INFO:root:2021/07/02 is done
INFO:root:2021/07/05 is done
INFO:root:2021/07/06 is done
INFO:root:2021/07/07 is done
INFO:root:2021/07/08 is done
INFO:root:2021/07/09 is done
INFO:root:2021/07/12 is done
INFO:root:2021/07/13 is done
INFO:root:2021/07/14 is done
INFO:root:2021/07/15 is done
INFO:root:2021/07/16 is done


In [48]:
cols = ['外資買賣超股數', '投信買賣超股數', '自營商買賣超股數', '自營商買賣超股數(避險)', '三大法人買賣超股數']
for row in pdf.iterrows():
    for c in cols:
        if row[1][c] == None:
            print(row)
            break
    

In [53]:
pdf[pdf['三大法人買賣超股數'].isna()]

,股票代號,名稱,外資買賣超股數,投信買賣超股數,自營商買賣超股數,自營商買賣超股數(避險),三大法人買賣超股數


In [18]:
row[1].

股票代號              1240
名稱                茂生農經
外資買賣超股數          -2000
投信買賣超股數              0
自營商買賣超股數             0
自營商買賣超股數(避險)         0
三大法人買賣超股數       -2,000
Name: 98, dtype: object

In [28]:
json_obj = json.loads(twse_resp.text)

In [33]:
json_obj = json.loads(twse_resp.text)
twse_data = json_obj['data']
twse_header = json_obj['fields']
twse_pdf = pd.DataFrame(twse_data, columns=twse_header)
# twse_pdf = twse_pdf[twse_pdf['證券代號'].str.len() == 4]
# twse_pdf = twse_pdf.rename(columns={'證券代號':'股票代號', '證券名稱':'名稱', '外陸資買賣超股數(不含外資自營商)': '外資買賣超股數'})
# twse_pdf = twse_pdf[header_list]

In [35]:
twse_pdf[twse_pdf['證券代號']=='1417']

,證券代號,證券名稱,外陸資買進股數(不含外資自營商),外陸資賣出股數(不含外資自營商),外陸資買賣超股數(不含外資自營商),外資自營商買進股數,外資自營商賣出股數,外資自營商買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
95,1417,嘉裕,"916,000","25,000","891,000",0,0,0,0,0,0,0,0,0,0,"891,000",None,None,None


In [50]:
pdf

,股票代號,名稱,外資買賣超股數,投信買賣超股數,自營商買賣超股數,自營商買賣超股數(避險),三大法人買賣超股數
98,1240,茂生農經,-2000.0,0.0,0.0,0,"-2,000"
99,1264,德麥,0.0,0.0,0.0,0,0
100,1268,漢來美食,1000.0,0.0,0.0,0,"1,000"
101,1336,台翰,65000.0,0.0,1.0,0,"65,001"
102,1565,精華,29700.0,17000.0,-4024.0,"-4,000","42,676"
...,...,...,...,...,...,...,...
1069,2883,開發金,19796546.0,-33396000.0,195000.0,"43,000","-13,404,454"
1070,6116,彩晶,-13524894.0,12000.0,-643592.0,"-505,000","-14,156,486"
1072,2618,長榮航,-17962500.0,-527000.0,593958.0,"633,000","-17,895,542"
1073,3481,群創,-47677583.0,23956000.0,-3159000.0,"-2,509,000","-26,880,583"


In [52]:
tmp_pdf = pdf[pdf['三大法人買賣超股數'].isna()].copy()
pdf.loc[pdf['三大法人買賣超股數'].isna(), '三大法人買賣超股數'] = tmp_pdf['外資買賣超股數'] + tmp_pdf['投信買賣超股數'] + tmp_pdf['自營商買賣超股數']
pdf.loc[(pdf['自營商買賣超股數(避險)'].isna()) & (pdf['自營商買賣超股數']==0), '自營商買賣超股數(避險)'] = 0

In [47]:
print(d)

['2409', '友達            ', '8,954,400', '67,324,395', '-58,369,995', '0', '0', '0', '26,554,000', '0', '26,554,000', '-3,049,433', '1,568,000', '2,283,433', '-715,433', '864,000', '3,198,000', '-2,334,000', '-34,865,428']
